# Setup 

In [ ]:
import os
import pandas as pd
from tecton_gen_ai.fco import source_as_knowledge, prompt, AgentService, AgentClient
from tecton_gen_ai.testing import make_local_source, set_dev_mode
from tecton_gen_ai.testing.utils import make_local_vector_db_config

set_dev_mode()

# Setup Sample Feature Views

In [ ]:
from tecton_gen_ai.testing import make_local_batch_feature_view

# user specific money transfer statistics
transfer_stats = make_local_batch_feature_view(
    "transfer_stats",
    [
        {"user_id": 1, "transfers_in_last_7_days": 20, "transfers_in_last_1_year": 22},
        {"user_id": 2, "transfers_in_last_7_days": 0, "transfers_in_last_1_year": 10},
    ],
    ["user_id"],
    description = "User's money transfer stats, abnormal activities could cause account suspension"  # description for LLM
)

# user profile info
user_profile = make_local_batch_feature_view(
    "user_profile",
    [
        {"user_id": 1, "name": "Jim", "age": 30, "account_status":"suspended"},
        {"user_id": 2, "name": "Mary", "age": 16, "account_status":"active"},
    ],
    ["user_id"],
    description = "User's name, age and account status"  # description for LLM
)


## The FAQ Data

As expected the FAQ data is text with common user questions and answers.

In [ ]:
faq_df = pd.read_parquet("faq.parquet")
display (faq_df)

## Prepare FAQ knowledge for RAG

The FAQ text is loaded into a vector search database to provide answers to user's questions.

In [ ]:
# The source description tells the LLM what kind of information it can retrieve from this knowledge base
faq_parquet_data = make_local_source(
    "faq",
    faq_df,
    description = "FAQ for TransferApp users",  # <<<<<<<<
    max_rows = len(faq_df)
)

# calculate embeddings based on the "question" such that LLM can find info it needs based on similar questions
faq_knowledge = source_as_knowledge(
    faq_parquet_data,  # << parquet file of questions and answers
    vector_db_config=make_local_vector_db_config(),
    vectorize_column="question" # <<<<<<<<
)

## Connect to LLM Service

In [ ]:
from tecton_gen_ai.testing.interactive import auto_complete, qna
from langchain_openai import ChatOpenAI

openai = ChatOpenAI(model = "gpt-4o-2024-08-06", temperature=0)


# Demo Scenario

TransferApp is a money transfer company that provides services through a mobile app.  

- They have an FAQ that user's can search to find responses to their questions.
- They want to create a chatbot that can use FAQ to respond to user questions directly.
- They've decided to build a RAG GenAI solution for this purpose.


In this demo of Tecton we will show how to build a chatbot using the Tecton GenAI Package.

- Tecton GenAI SDK to build two version of the chatbot: 
    - RAG Solution
    - Personalized RAG Solution
- We will use Tecton's declarative framework to create:
    - prompts 
    - knowledge base from the FAQ data
    - user features as LLM tools to create a personalized experience
    - Agent Service to serve the chatbot functionality in TransferApp


# RAG Solution Options

<img src="images/naive_rag.png" width="1000"/>

## With Tecton

<img src="images/rag_diagram2.png" width="1000"/>

## Managed RAG Chatbot with Tecton
- The prompt provides instructions to the LLM
- Prepare the FAQ Knowledge base
- The AgentService serves the GenAI application through a REST endpoint:
    - the prompt
    - the FAQ knowledge 
    - manages the LLM interaction


In [ ]:
# the prompt instructs the LLM to use the FAQ knowledge to answer questions 
@prompt()
def sys_prompt() -> str:
    return """You are an assistant,
    TransferApp is a money transfer service on a mobile application.
    You answer questions based on FAQ for TranferApp.
    Say you don't know if the questions are not relevant to any questions on FAQ
"""

# calculate embeddings based on the "question" such that LLM can find info it needs based on similar questions
faq_knowledge = source_as_knowledge(
    faq_parquet_data,  # << parquet file of questions and answers
    vector_db_config=make_local_vector_db_config(),
    vectorize_column="question" # <<<<<<<<
)

rag_service = AgentService(
    "chatbot_context",
    prompts=[sys_prompt],
    knowledge=[faq_knowledge]
)

rag_client = AgentClient.from_local(rag_service)

## A Simple RAG chatbot

It uses generally applicable knowledge so all users get the same experience.

In [ ]:
qna(rag_client, openai, "sys_prompt")

 # Adding Customized Context to the Chatbot

A more useful chatbot can be created by adding better context and tools for the LLM.

<img src=images/personalized_diagram.png width=1000/>


## Building a smarter TransferApp chatbot

In this example we use two feature pipelines:
 - **transfer_stats** transfer activity aggregations `transfers_last_7days` & `transfers_last_1year`
 - **user_profile**  data containing `name`, `age` & `account_status`

We build a contextualized prompt to provide a personalized experience.

We build an LLM Agent that delivers: 
- FAQ based knowledge
- contextualized prompt 
- tools to access user specific data : **transfer_stats** & **user_profile** data

In [ ]:
from tecton_gen_ai.fco import tool


# this prompt incorporates user_profile context in the prompt  
@prompt(sources=[user_profile])
def sys_prompt_fv(user_profile) -> str:
    return f"""
            You are an assistant, 
            You answer questions based on the FAQ for TransferApp.
            Say you don't know if the questions are not relevant to any questions on FAQ.
            You are serving {user_profile['name']}, whose age is {user_profile['age']}.
            Address the user by name.
            Consult the user's money transfer stats if needed to respond to their question.
    """
    
# a service with user context for personalization
service_with_fv = AgentService(
    "chatbot_with_personalization",
    prompts=[sys_prompt_fv],
    knowledge=[faq_knowledge],
    tools=[transfer_stats, user_profile],  # <<<<<< the LLM can now query this user info as needed
)

client_with_fv = AgentClient.from_local(service_with_fv)

## A chatbot with user context

The chatbot identifies the user with a `user_id` based on their session.

Tecton makes use of the `user_id` to personalize the interaction.

In [ ]:
# the chatbot has the user identified based on their session
qna(client_with_fv, openai, "sys_prompt_fv", context={"user_id":2}) 

# Tecton Apply

`tecton apply` command will deploy:
- feature views, 
- prompts, 
- knowledge bases 
- agents

What happens when you `tecton apply`:

- Data Pipelines are automatically created, scheduled, orchestrated and monitored 
- Data is kept up to date, for streaming and batch sources
- Tecton's Retrieval system provide low-latency serving of the features and agent services
- Tecton deploys API endpoint for the AgentService that 
    - controls authenticated access to the chatbot / genai app
    - auto-scales
    - real-time inputs and their processing logic is also deployed

Output from `tecton apply`:

<img src="images/tecton-apply.png" width="800"/>



# Conclusions

Tecton's Declarative Framework makes it easy to build Generative AI applications
- prompts, knowledge bases and personalized tools are just a few lines of code
- iterating to do prompt engingeering is fast
- knowledge base management is automatic
- incorporating personalization through enriched prompts and features as tools is simple

The Tecton Platform gets the GenAI app into production fast by automating:
- data engineering 
- versioning
- data lineage
- governance
- orchestration
- scheduling
- monitoring

You focus on the Generative AI behavior you want to produce. Tecton takes care of the rest.